In [ ]:
# Imported packages.

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import ode

In [ ]:
# Constants for the problem in cgs units
G = 6.67e-8 # Gravitational constant
M_s = 1.98e33 # Mass of Sun
M_e = 5.97e27 # Mass of Earth
m_m = 7.349e25 # Mass of Moon
R_e = 6.371e8 # Radius of Earth
k2 = 0.298 # Lover number of Earth
Q_m = 11.5 # Tidal quality factor
a0 = 3.84e10 # Present day value of the Lunar semimajoraxis
a_e = 1.49e13 # Semimajor axis of Earth’s orbit
I = 0.3299*M_e*R_e**2 # Earth’s moment of inertia
W = (2*np.pi)/86164 # Angular velocity of Earth

# Write out the system of equations
def T_moon(a):
    T_m = 3/2 * G*m_m**2/a * (R_e/a)**5 * (k2/Q_m)
    return T_m

def T_sun(a):
    T_s = 1/4.7 * (a/a0)**6 * T_moon(a)
    return T_s

def EMsys(t, Z):
    L_e, S_e, L_m = Z
    a = (L_m/m_m)**2 / (G*(M_e+m_m))
    f = [T_sun(a), -T_sun(a)-T_moon(a), T_moon(a)]
    return f

# Choose solver
solver = ode(EMsys)
solver.set_integrator('dopri5')

# Set the initial conditions
t0 = 0.0
L_e0, S_e0, L_m0 = M_e*np.sqrt(G*(M_s+M_e)*a_e), I*W, m_m*np.sqrt(G*(M_e+m_m)*a0)
Z0 = [L_e0, S_e0, L_m0]

solver.set_initial_value(Z0, t0)

T = -6.31152e16 # Final time is 2 billion years in the past
n = 300

# Create an empty array to store the solution
t = np.linspace(t0, T, n)
sol = np.empty((n, 3))
sol[0] = Z0

i = 1
while solver.successful() and solver.t > T:
    solver.integrate(t[i])
    sol[i] = solver.y
    i += 1


In [ ]:
print("These are the present day values for L_e, S_e, L_m in cgs unit system")
Z0

In [ ]:
T_m = 3/2 * G*m_m**2/a0 * (R_e/a0)**5 * (k2/Q_m)
T_s = 1/4.7 * (a0/a0)**6*T_m

print("This is the present day Tmoon value,", T_m)
print("This is the present day Tsun value", T_s)

In [ ]:
tau_Ls = L_e0/T_s
tau_Se = S_e0/(T_s + T_m)
tau_Lm = L_m0/T_m
print(tau_Ls, tau_Se, tau_Lm)

In [ ]:
plt.plot(t/60/60/24/365/10**9, (sol[:,2]/m_m)**2 / (G*(M_e+m_m)) / 10**5, label='Semimajor axis of Moon')
#plt.plot(t, sol[:,0], label='L_e', color = 'limegreen')
#plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
#plt.plot(t, sol[:,1], label='S_e', color = 'royalblue')
#plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
#plt.plot(t, sol[:,2], label='L_m', color = 'tomato')
#plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
plt.xlabel('Time [$10^{9} years$]')
plt.ylabel('$a_{moon}$ [$kms$]')
plt.title('Semimajor axis of the Moon as time go back 2 billion years')
plt.legend()
plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
plt.tight_layout()
plt.savefig("plot1.pdf", dpi=100)
plt.show()

In [ ]:
# Determining at which index a_moon goes to zero
np.where((sol[:,2]/m_m)**2 / (G*(M_e+m_m)) < 1.0e-10)

In [ ]:
# Retriving how long ago the moon formed 
print('The moon formed',abs(t[230]/60/60/24/365/10**9), 'billion years ago')

In [ ]:
# Plotting the Length of Day on Earth vs. time
plt.plot(t/60/60/24/365/10**9, (86164*S_e0)/(sol[:,1]) / (3600), label='Length of Day on Earth')
plt.xlabel('Time [$10^{9} years$]')
plt.ylabel('$LoD$ [$hrs.$]')
plt.title('Length of Day on Earth'
          '\n as Time goes back 2 billion years')
plt.legend()
plt.grid(b=True, color='DarkTurquoise', alpha=0.2, linestyle=':', linewidth=2)
plt.tight_layout()
plt.savefig("plot2.pdf")
plt.show()

In [ ]:
print('This is was the length of day when the Earth and Moon collided', (86164*S_e0)/(sol[:,1][229]) /3600, 'hrs.')

In [ ]:
# Determining at which index the semimajor axis of the Moon is at the Roche Radius
#Roche Radius = 9492 km
np.where((sol[:,2]/m_m)**2 / (G*(M_e+m_m)) > 949200000)

In [ ]:
# Retriving the Length of Day on Earth when the Moon was at the Roche Radius 
print('When the moon is at Roche Radius, the length of day on Earth is', (86164*S_e0)/(sol[:,1][229]) /3600, 'hrs.')

In [ ]:
# Printing the age of the Moon and Earth
print('Age of Moon is 4.51 billion years')
print('Age of Earth is 4.54 billion years')